In [ ]:
import sys
import heapq

## State-space tree (상태공간트리)
##### 1. Backtracking
##### 2. Branch-and-bound
##### 3. A* algorithm

#### 1. Backtracking
##### - DFS 또는 그와 유사한 스타일의 탐색을 총칭
##### - 가능한 지점까지 탐색하다가 막히면 되돌아간다

#### (1) Maze 문제

In [ ]:
Maze={'a':['c'], #start
      'b':['c'],
      'c':['b','e'],
      'd':['e'],
      'e':['f'],
      'f':['e','g','h'],
      'g':['f'],
      'h':['f','i'],
      'i':['h'], #target
      'j':['k'],
      'k':['h','j','l','m'],
      'm':['k']
      }

In [ ]:
def get_path(v):
  print(v,end=' > ')
  if prev[v]!=v:
    get_path(prev[v])
def maze1(v,T='i'): # ppt 방식
  visited[v]=True
  print(v)
  if v==T:
    print('성공!')
    get_path(v)
    sys.exit()
  for x in Maze[v]:
    if visited[x]==False:
      prev[x]=v
      maze1(x)
visited={vertex:False for vertex in Maze}
prev={vertex:vertex for vertex in Maze}
maze1('a')
print(prev)

a
c
b
e
f
g
h
i
성공!
i > h > f > e > c > a > 

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
def get_path(v):
  print(v,end="")
  if prev[v]!=v:
    print(" > ",end="")
    get_path(prev[v])

def maze2(v,T='i'): # 내 방식
  print(v)
  if v==T:
    print("성공!")
    get_path(v)
    sys.exit()
  else:
    for x in Maze[v]:
      if visited[x]==False:
        visited[x]=True
        prev[x]=v
        maze2(x)
        visited[x]=False
visited={vertex:False for vertex in Maze}
prev={vertex:vertex for vertex in Maze}
maze2('a')

a
c
b
e
f
g
h
i
성공!
i > h > f > e > c > a

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#### (2) Coloring 문제
##### - 그래프에서 인접한 vertex는 같은 색을 칠할 수 없다고 할 때, k개의 색상을 사용해서 전체 graph를 칠할 수 있는가?

In [ ]:
Map={1:[2,3,4],
     2:[1,5],
     3:[1,6],
     4:[1,5,6],
     5:[2,4,6],
     6:[1,2,3,4,5]}

In [ ]:
def valid(i,c):
  for j in range(1,i):
    if j in Map[i] and color[j]==c:
      return False
  return True
def kColoring(i,c):
  if valid(i,c):
    color[i]=c
    if i==len(Map):
      return True
    else:
      result=False
      d=1
      while result==False and d<=k:
        result=kColoring(i+1,d)
        d+=1
      return result
  else:
    return False
for k in range(1,7):
  color={}
  print(f'색상의 수:{k}\n가능 여부: {kColoring(1,1)}')

색상의 수:1
가능 여부: False
색상의 수:2
가능 여부: False
색상의 수:3
가능 여부: True
색상의 수:4
가능 여부: True
색상의 수:5
가능 여부: True
색상의 수:6
가능 여부: True


#### 2. Branch-and-Bound
##### - Branch(분기)와 Bound(한정)을 결합
##### - 분기를 한정시켜 쓸데 없는 시간 낭비를 줄임
##### - Backtracking과 비교
###### 공통점: 경우들을 차례로 나열하는 방법
###### 차이점: Backtracking은 탐색하다가 더 이상 진행이 되지 않으면 되돌아가지만, Branch-and-Bound는 최적해를 찾을 가능성이 없으면 분기는 하지 않는 방식
##### - 부모 노드의 자식 들 중 평가 함수의 하한이 가장 낮은 것(최소값 문제)을 새로운 부모 노드로 두고 지역 최적 값이 등장하면 이보다 하한 값이 큰 노드들은 분기하지 않음

#### (1) Branch and Bound 기반 TSP 문제

In [ ]:
INF=sys.maxsize
TSP_map=[[INF,10,10,30,25],
         [10,INF,14,21,10],
         [10,18,INF,7,9],
         [8,11,7,INF,3],
         [14,10,10,3,INF]]

In [ ]:
def Branch_and_Bound(r):
  heap=[]
  heapq.heappush(heap,(sum([min(TSP_map[i]) for i in range(len(TSP_map))]),[r]))
  func=0
  while heap:
    tmp=heapq.heappop(heap)
    if len(TSP_map)==len(tmp[1]):
      func=tmp[0]
      while heap:
        bound=heapq.heappop(heap)
        if bound[0]<=func:
          heapq.heappush(heap,bound)
          break
    for i in range(len(TSP_map)):
      if i not in tmp[1] or (len(tmp[1])==len(TSP_map) and i==0):
        heapq.heappush(heap,(tmp[0]-min(TSP_map[tmp[1][-1]])+TSP_map[tmp[1][-1]][i],tmp[1]+[i]))
  return tmp

start=0
result=Branch_and_Bound(start)
print(f'최소값: {result[0]}\n경로: {result[1]}')

최소값: 40
경로: [0, 1, 4, 3, 2, 0]


#### 3. A* Algorithm
##### #Best-First Search: 각 정점이 매력함수 값 g(x)을 가지며 방문하지 않은 정점들 중 매력함수 값이 가장 작은 것부터 방문
##### - A* Algorithm은 Best-First Search에 목적점까지의 잔여추정거리 h(x)를 추가로 고려하는 알고리즘
##### - 방문하지 않은 정점들 중 f(x)=g(x)+h(x) 값이 가장 작은 순으로 방문
##### 이 때 정점 x부터 목적점에 이르는 잔여거리 h(x)는 실제 거리보다 커서는 안됨

#### (1) A* 기반 다익스트라 
##### - 각 위치에서 행선지로의 추정 잔여 거리를 입력으로 받아야함

In [ ]:
Graph3={'a':{'b':20},
       'b':{'c':17},
       'e':{'i':39,'h':25},
       'c':{'a':10,'b':17,'d':30,'e':25,'f':23},
       'd':{'a':19,'g':24},
       'f':{'d':16,'e':28,'g':18},
       'g':{'j':20},
       'h':{'i':29},
       'i':{'j':28},
       'j':{'h':40}}

In [ ]:
pred_Distance_x_to_T={'a':{'j':61},
                      'b':{'j':68},
                      'c':{'j':52},
                      'd':{'j':40},
                      'e':{'j':52},
                      'f':{'j':34},
                      'g':{'j':19},
                      'h':{'j':39},
                      'i':{'j':19},
                      'j':{'j':0},
                      }

In [ ]:
def A_star_Dijkstra(start,T):
  Dist={vertex : sys.maxsize for vertex in Graph3}
  Dist[start]=0
  heap=[]
  heapq.heappush(heap,(Dist[start]+pred_Distance_x_to_T[start][T],Dist[start],start))
  while heap:
    tmp=heapq.heappop(heap)
    print(tmp)
    if tmp[2]==T:
      return tmp[1]
    if tmp[1]>Dist[tmp[2]]:
      continue
    for next_destnation in Graph3[tmp[2]]:
      if Dist[next_destnation]>Dist[tmp[2]]+Graph3[tmp[2]][next_destnation]:
        Dist[next_destnation]=Dist[tmp[2]]+Graph3[tmp[2]][next_destnation]
        heapq.heappush(heap,(Dist[next_destnation]+pred_Distance_x_to_T[next_destnation][T],Dist[next_destnation],next_destnation))

A_star_Dijkstra('c','j')


(52, 0, 'c')
(57, 23, 'f')
(60, 41, 'g')
(61, 61, 'j')


61